In [1]:
## Import Libraries

import pandas as pd
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [2]:
## Read Files

gdf_lsoa = gpd.read_file("Data/manchester_lsoa/manchester_lsoa.shp")

df_pop = pd.read_csv("Data/population_lsoa.csv", skiprows=3, low_memory=False)

df_car_disabled = pd.read_csv("Data/car_disabled_lsoa.csv")

df_annual_income = pd.read_csv("Data/net_annual_income.csv", skiprows=4)

df_lookup = pd.read_csv("Data/msoa_lsoa_lookup.csv")

df_deprived = pd.read_csv("Data/deprivation_household.csv", skiprows=6)

## Populations

In [3]:
## Filter only population within Greater Manchester

pop_lsoa = df_pop[df_pop["LSOA 2021 Code"].isin(gdf_lsoa["lsoa21cd"].unique())]
pop_lsoa["Total"] = pop_lsoa["Total"].str.replace(",", "").astype(int)

/tmp/ipykernel_15271/1361827563.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["Total"] = pop_lsoa["Total"].str.replace(",", "").astype(int)


In [4]:
pop_lsoa

,LAD 2021 Code,LAD 2021 Name,LSOA 2021 Code,LSOA 2021 Name,Total,F0,F1,F2,F3,F4,...,M81,M82,M83,M84,M85,M86,M87,M88,M89,M90
530,E06000007,Warrington,E01012472,Warrington 001B,1865,0,14,5,5,10,...,5,11,6,11,4,6,0,0,7,4
533,E06000007,Warrington,E01012475,Warrington 002C,1495,6,5,10,5,5,...,5,5,6,6,0,6,6,0,0,5
534,E06000007,Warrington,E01012476,Warrington 001C,1578,7,5,10,5,5,...,5,5,6,0,6,0,6,7,0,5
535,E06000007,Warrington,E01012477,Warrington 001D,1614,3,5,5,10,5,...,5,5,6,6,6,6,0,0,0,10
573,E06000007,Warrington,E01012517,Warrington 021E,2340,12,14,10,10,15,...,5,5,5,5,0,0,7,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23612,E08000013,St. Helens,E01006885,St. Helens 001D,1327,0,5,5,5,5,...,5,4,0,5,5,0,0,7,0,5
27554,E08000033,Calderdale,E01010886,Calderdale 022C,1121,5,10,5,0,5,...,5,0,6,6,0,6,0,0,0,0
27623,E08000033,Calderdale,E01010955,Calderdale 027D,1400,8,5,5,10,5,...,0,5,0,0,0,0,0,0,0,0
27743,E08000034,Kirklees,E01011075,Kirklees 052C,1452,3,0,5,5,10,...,5,0,0,5,5,6,6,6,0,5


### Young Adult Women Population (18-29 years old)

In [5]:
## Checking the columns for female aged 18-29
print(pop_lsoa.columns[23:35])

## Checking data types and any outlier
print(pop_lsoa[list(pop_lsoa.columns[23:35])].dtypes)
print(pop_lsoa["F19"].unique())

Index(['F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27',
       'F28', 'F29'],
      dtype='object')
F18     int64
F19    object
F20     int64
F21     int64
F22     int64
F23     int64
F24     int64
F25     int64
F26     int64
F27     int64
F28     int64
F29     int64
dtype: object
['5' '0' '11' '10' '9' '4' '6' '16' '21' '31' '26' '36' '14' '19' '15'
 '133' '29' '24' '49' '468' '162' '414' '424' '123' '98' '473' '192' '79'
 '39' '93' '384' '74' '59' '296' '167' '197' '500' '104' '41']


In [6]:
## Convert F19 column to numeric
pop_lsoa["F19"] = pop_lsoa["F19"].apply(pd.to_numeric, errors='coerce')

/tmp/ipykernel_15271/3735554503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["F19"] = pop_lsoa["F19"].apply(pd.to_numeric, errors='coerce')


In [7]:
## Create a list of columns for young adult women
women_column = list(pop_lsoa.columns[23:35])

## Create a new column 'women' that sums all these columns
pop_lsoa["women_pop"] = pop_lsoa[women_column].sum(axis=1)

## Create a new column 'women_percentage' that calculates the percentage of 'women' over 'Total'
pop_lsoa["women_percentage"] = pop_lsoa["women_pop"] / pop_lsoa["Total"] * 100
pop_lsoa

/tmp/ipykernel_15271/613156241.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["women_pop"] = pop_lsoa[women_column].sum(axis=1)
/tmp/ipykernel_15271/613156241.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["women_percentage"] = pop_lsoa["women_pop"] / pop_lsoa["Total"] * 100


,LAD 2021 Code,LAD 2021 Name,LSOA 2021 Code,LSOA 2021 Name,Total,F0,F1,F2,F3,F4,...,M83,M84,M85,M86,M87,M88,M89,M90,women_pop,women_percentage
530,E06000007,Warrington,E01012472,Warrington 001B,1865,0,14,5,5,10,...,6,11,4,6,0,0,7,4,81,4.343164
533,E06000007,Warrington,E01012475,Warrington 002C,1495,6,5,10,5,5,...,6,6,0,6,6,0,0,5,60,4.013378
534,E06000007,Warrington,E01012476,Warrington 001C,1578,7,5,10,5,5,...,6,0,6,0,6,7,0,5,80,5.069708
535,E06000007,Warrington,E01012477,Warrington 001D,1614,3,5,5,10,5,...,6,6,6,6,0,0,0,10,50,3.097893
573,E06000007,Warrington,E01012517,Warrington 021E,2340,12,14,10,10,15,...,5,5,0,0,7,0,0,4,108,4.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23612,E08000013,St. Helens,E01006885,St. Helens 001D,1327,0,5,5,5,5,...,0,5,5,0,0,7,0,5,45,3.391108
27554,E08000033,Calderdale,E01010886,Calderdale 022C,1121,5,10,5,0,5,...,6,6,0,6,0,0,0,0,65,5.798394
27623,E08000033,Calderdale,E01010955,Calderdale 027D,1400,8,5,5,10,5,...,0,0,0,0,0,0,0,0,65,4.642857
27743,E08000034,Kirklees,E01011075,Kirklees 052C,1452,3,0,5,5,10,...,0,5,5,6,6,6,0,5,80,5.509642


### Elderly Population (> 64 years old)

In [8]:
## Checking the columns for female 65+ and male 65+
print(pop_lsoa.columns[70:96])
print(pop_lsoa.columns[161:])

Index(['F65', 'F66', 'F67', 'F68', 'F69', 'F70', 'F71', 'F72', 'F73', 'F74',
       'F75', 'F76', 'F77', 'F78', 'F79', 'F80', 'F81', 'F82', 'F83', 'F84',
       'F85', 'F86', 'F87', 'F88', 'F89', 'F90'],
      dtype='object')
Index(['M65', 'M66', 'M67', 'M68', 'M69', 'M70', 'M71', 'M72', 'M73', 'M74',
       'M75', 'M76', 'M77', 'M78', 'M79', 'M80', 'M81', 'M82', 'M83', 'M84',
       'M85', 'M86', 'M87', 'M88', 'M89', 'M90', 'women_pop',
       'women_percentage'],
      dtype='object')


In [9]:
# Create new columns for elderly counts and percentage

## Create a list of columns for elderly male and female
elderly_f = pop_lsoa.columns[70:96]
elderly_m = pop_lsoa.columns[161:]

elderly_column = list(elderly_f) + list(elderly_m)

## Create a new column 'Population 65+' that sums all these columns
pop_lsoa["elderly_pop"] = pop_lsoa[elderly_column].sum(axis=1)

## Create a new column 'Elderly Percentage' that calculates the percentage of 'Population 65+' over 'Total'
pop_lsoa["elderly_percentage"] = pop_lsoa["elderly_pop"] / pop_lsoa["Total"] * 100
pop_lsoa

/tmp/ipykernel_15271/1869302055.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["elderly_pop"] = pop_lsoa[elderly_column].sum(axis=1)
/tmp/ipykernel_15271/1869302055.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["elderly_percentage"] = pop_lsoa["elderly_pop"] / pop_lsoa["Total"] * 100


,LAD 2021 Code,LAD 2021 Name,LSOA 2021 Code,LSOA 2021 Name,Total,F0,F1,F2,F3,F4,...,M85,M86,M87,M88,M89,M90,women_pop,women_percentage,elderly_pop,elderly_percentage
530,E06000007,Warrington,E01012472,Warrington 001B,1865,0,14,5,5,10,...,4,6,0,0,7,4,81,4.343164,555.343164,29.777113
533,E06000007,Warrington,E01012475,Warrington 002C,1495,6,5,10,5,5,...,0,6,6,0,0,5,60,4.013378,392.013378,26.221631
534,E06000007,Warrington,E01012476,Warrington 001C,1578,7,5,10,5,5,...,6,0,6,7,0,5,80,5.069708,476.069708,30.169183
535,E06000007,Warrington,E01012477,Warrington 001D,1614,3,5,5,10,5,...,6,6,0,0,0,10,50,3.097893,365.097893,22.620687
573,E06000007,Warrington,E01012517,Warrington 021E,2340,12,14,10,10,15,...,0,0,7,0,0,4,108,4.615385,496.615385,21.222880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23612,E08000013,St. Helens,E01006885,St. Helens 001D,1327,0,5,5,5,5,...,5,0,0,7,0,5,45,3.391108,442.391108,33.337687
27554,E08000033,Calderdale,E01010886,Calderdale 022C,1121,5,10,5,0,5,...,0,6,0,0,0,0,65,5.798394,334.798394,29.866048
27623,E08000033,Calderdale,E01010955,Calderdale 027D,1400,8,5,5,10,5,...,0,0,0,0,0,0,65,4.642857,345.642857,24.688776
27743,E08000034,Kirklees,E01011075,Kirklees 052C,1452,3,0,5,5,10,...,5,6,6,6,0,5,80,5.509642,443.509642,30.544741


### Children Population (< 17 years old)

In [10]:
## Checking the columns for female 16- and male 16-
print(pop_lsoa.columns[5:22])
print(pop_lsoa.columns[96:113])

Index(['F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16'],
      dtype='object')
Index(['M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10',
       'M11', 'M12', 'M13', 'M14', 'M15', 'M16'],
      dtype='object')


In [11]:
# Create new columns for children counts and percentage

## Create a list of columns for children male and female

children_f = pop_lsoa.columns[5:22]
children_m = pop_lsoa.columns[96:113]

children_column = list(children_f) + list(children_m)

## Create a new column 'Population 65+' that sums all these columns
pop_lsoa["children_pop"] = pop_lsoa[children_column].sum(axis=1)

## Create a new column 'Elderly Percentage' that calculates the percentage of 'Population 65+' over 'Total'
pop_lsoa["children_percentage"] = pop_lsoa["children_pop"] / pop_lsoa["Total"] * 100
pop_lsoa

/tmp/ipykernel_15271/469105569.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["children_pop"] = pop_lsoa[children_column].sum(axis=1)
/tmp/ipykernel_15271/469105569.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_lsoa["children_percentage"] = pop_lsoa["children_pop"] / pop_lsoa["Total"] * 100


,LAD 2021 Code,LAD 2021 Name,LSOA 2021 Code,LSOA 2021 Name,Total,F0,F1,F2,F3,F4,...,M87,M88,M89,M90,women_pop,women_percentage,elderly_pop,elderly_percentage,children_pop,children_percentage
530,E06000007,Warrington,E01012472,Warrington 001B,1865,0,14,5,5,10,...,0,0,7,4,81,4.343164,555.343164,29.777113,354,18.981233
533,E06000007,Warrington,E01012475,Warrington 002C,1495,6,5,10,5,5,...,6,0,0,5,60,4.013378,392.013378,26.221631,274,18.327759
534,E06000007,Warrington,E01012476,Warrington 001C,1578,7,5,10,5,5,...,6,7,0,5,80,5.069708,476.069708,30.169183,257,16.286439
535,E06000007,Warrington,E01012477,Warrington 001D,1614,3,5,5,10,5,...,0,0,0,10,50,3.097893,365.097893,22.620687,307,19.021066
573,E06000007,Warrington,E01012517,Warrington 021E,2340,12,14,10,10,15,...,7,0,0,4,108,4.615385,496.615385,21.222880,557,23.803419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23612,E08000013,St. Helens,E01006885,St. Helens 001D,1327,0,5,5,5,5,...,0,7,0,5,45,3.391108,442.391108,33.337687,192,14.468726
27554,E08000033,Calderdale,E01010886,Calderdale 022C,1121,5,10,5,0,5,...,0,0,0,0,65,5.798394,334.798394,29.866048,215,19.179304
27623,E08000033,Calderdale,E01010955,Calderdale 027D,1400,8,5,5,10,5,...,0,0,0,0,65,4.642857,345.642857,24.688776,265,18.928571
27743,E08000034,Kirklees,E01011075,Kirklees 052C,1452,3,0,5,5,10,...,6,6,0,5,80,5.509642,443.509642,30.544741,251,17.286501


In [12]:
## Drop unnecessary columns

merge_lsoa = gdf_lsoa.drop(columns = ["label", "name"])

In [13]:
## Calculate area of each LSOA in square meters

merge_lsoa["area_sqkm"] = merge_lsoa["geometry"].area / 10**6

In [14]:
## Merge total population, 65+ percentage, 16- percentage to the gdf

merge_lsoa = pd.merge(merge_lsoa, pop_lsoa[["LSOA 2021 Code", "Total", "women_percentage", "elderly_percentage", "children_percentage"]], left_on = "lsoa21cd", right_on = "LSOA 2021 Code")

In [15]:
## Drop redundant column

merge_lsoa = merge_lsoa.drop(columns=["LSOA 2021 Code"])

In [16]:
merge_lsoa

,lsoa21cd,lsoa21nm,geometry,area_sqkm,Total,women_percentage,elderly_percentage,children_percentage
0,E01004901,Bolton 028E,"POLYGON ((366003.093 406666.688, 366004.412 40...",0.354363,1773,5.922166,29.606439,15.172025
1,E01005001,Bury 012B,"POLYGON ((375480.348 408593.123, 375481.063 40...",0.238302,1228,4.885993,22.384853,19.218241
2,E01004840,Bolton 012A,"POLYGON ((367374.781 411954.594, 367387.095 41...",4.181101,1769,4.578858,30.275798,19.106840
3,E01005006,Bury 012D,"POLYGON ((377212.688 410849.500, 377231.906 41...",1.772635,2051,6.874695,25.688674,20.234032
4,E01004773,Bolton 003D,"POLYGON ((371776.559 412463.204, 371825.243 41...",0.266227,1985,7.103275,25.395631,19.143577
...,...,...,...,...,...,...,...,...
1763,E01006394,Wigan 021B,"POLYGON ((355212.000 403394.000, 355230.000 40...",0.592787,1535,4.560261,34.043014,18.045603
1764,E01006410,Wigan 020E,"POLYGON ((357596.954 403017.375, 357616.740 40...",0.628715,1261,4.758128,33.287718,15.543220
1765,E01006400,Wigan 021D,"POLYGON ((356213.760 403792.397, 356213.312 40...",0.569408,1982,6.004036,18.415945,26.135217
1766,E01006331,Wigan 031D,"POLYGON ((367336.970 401009.452, 367340.657 40...",0.596520,1501,3.997335,28.514146,16.122585


### Population Density

In [17]:
## Calculate population density

merge_lsoa["pop_density"] = merge_lsoa["Total"] / merge_lsoa["area_sqkm"]
merge_lsoa

,lsoa21cd,lsoa21nm,geometry,area_sqkm,Total,women_percentage,elderly_percentage,children_percentage,pop_density
0,E01004901,Bolton 028E,"POLYGON ((366003.093 406666.688, 366004.412 40...",0.354363,1773,5.922166,29.606439,15.172025,5003.350923
1,E01005001,Bury 012B,"POLYGON ((375480.348 408593.123, 375481.063 40...",0.238302,1228,4.885993,22.384853,19.218241,5153.129520
2,E01004840,Bolton 012A,"POLYGON ((367374.781 411954.594, 367387.095 41...",4.181101,1769,4.578858,30.275798,19.106840,423.094327
3,E01005006,Bury 012D,"POLYGON ((377212.688 410849.500, 377231.906 41...",1.772635,2051,6.874695,25.688674,20.234032,1157.034831
4,E01004773,Bolton 003D,"POLYGON ((371776.559 412463.204, 371825.243 41...",0.266227,1985,7.103275,25.395631,19.143577,7456.052936
...,...,...,...,...,...,...,...,...,...
1763,E01006394,Wigan 021B,"POLYGON ((355212.000 403394.000, 355230.000 40...",0.592787,1535,4.560261,34.043014,18.045603,2589.463159
1764,E01006410,Wigan 020E,"POLYGON ((357596.954 403017.375, 357616.740 40...",0.628715,1261,4.758128,33.287718,15.543220,2005.677534
1765,E01006400,Wigan 021D,"POLYGON ((356213.760 403792.397, 356213.312 40...",0.569408,1982,6.004036,18.415945,26.135217,3480.807917
1766,E01006331,Wigan 031D,"POLYGON ((367336.970 401009.452, 367340.657 40...",0.596520,1501,3.997335,28.514146,16.122585,2516.260480


## Disabled People - Car Ownership

In [18]:
df_car_disabled

,Lower layer Super Output Areas Code,Lower layer Super Output Areas,Number of disabled people in household (3 categories) Code,Number of disabled people in household (3 categories),Car or van availability (4 categories) Code,Car or van availability (4 categories),Observation
0,E01000001,City of London 001A,-8,Does not apply,-8,Does not apply,0
1,E01000001,City of London 001A,-8,Does not apply,0,No cars or vans in household,0
2,E01000001,City of London 001A,-8,Does not apply,1,1 car or van in household,0
3,E01000001,City of London 001A,-8,Does not apply,2,2 or more cars or vans in household,0
4,E01000001,City of London 001A,0,No people disabled under the Equality Act in h...,-8,Does not apply,0
...,...,...,...,...,...,...,...
428011,W01002040,Vale of Glamorgan 015F,0,No people disabled under the Equality Act in h...,2,2 or more cars or vans in household,264
428012,W01002040,Vale of Glamorgan 015F,1,1 or more people disabled under the Equality A...,-8,Does not apply,0
428013,W01002040,Vale of Glamorgan 015F,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,55
428014,W01002040,Vale of Glamorgan 015F,1,1 or more people disabled under the Equality A...,1,1 car or van in household,120


In [19]:
## Filter only rows which fulfil the condition of 1 or more disabled people in the household AND having no car
car_disabled = df_car_disabled[(df_car_disabled["Number of disabled people in household (3 categories) Code"] == 1) & (df_car_disabled["Car or van availability (4 categories) Code"] == 0)]

## Filter only in Greater Manchester
car_disabled = car_disabled[car_disabled["Lower layer Super Output Areas Code"].isin(merge_lsoa["lsoa21cd"].unique())]

In [20]:
car_disabled

,Lower layer Super Output Areas Code,Lower layer Super Output Areas,Number of disabled people in household (3 categories) Code,Number of disabled people in household (3 categories),Car or van availability (4 categories) Code,Car or van availability (4 categories),Observation
54081,E01004766,Bolton 005A,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,148
54093,E01004767,Bolton 005B,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,98
54105,E01004768,Bolton 001A,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,15
54117,E01004769,Bolton 003A,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,33
54129,E01004770,Bolton 003B,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,29
...,...,...,...,...,...,...,...
386145,E01034138,Manchester 064B,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,82
387237,E01034232,Oldham 036A,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,70
387249,E01034233,Oldham 036B,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,67
387261,E01034234,Rochdale 008H,1,1 or more people disabled under the Equality A...,0,No cars or vans in household,57


In [21]:
## Merge disabled-no car observation to the gdf

merge_lsoa = pd.merge(merge_lsoa, car_disabled[["Lower layer Super Output Areas Code", "Observation"]], left_on="lsoa21cd", right_on="Lower layer Super Output Areas Code")
merge_lsoa = merge_lsoa.rename(columns={"Observation": "disabled_no_car"})

In [22]:
merge_lsoa = merge_lsoa.drop(columns=["Lower layer Super Output Areas Code"])

## Deprived Households

In [23]:
df_deprived

,2021LSOANM,2021LSOACD,Total: All households,Household is deprived in one dimension,%
0,Hartlepool 001A,E01011954,963,304,31.6
1,Hartlepool 001B,E01011969,603,199,33.0
2,Hartlepool 001C,E01011970,488,170,34.8
3,Hartlepool 001D,E01011971,519,147,28.3
4,Hartlepool 001F,E01033465,740,209,28.2
...,...,...,...,...,...
35667,Merthyr Tydfil 008B,W01001300,694,225,32.4
35668,Merthyr Tydfil 008C,W01001301,770,230,29.9
35669,Merthyr Tydfil 008D,W01001308,484,157,32.4
35670,Merthyr Tydfil 008E,W01001309,518,168,32.4


In [24]:
## Filter only in Greater Manchester

deprived_lsoa = df_deprived[df_deprived["2021LSOACD"].isin(gdf_lsoa["lsoa21cd"].unique())]
deprived_lsoa

,2021LSOANM,2021LSOACD,Total: All households,Household is deprived in one dimension,%
1763,Warrington 001B,E01012472,751,237,31.6
1764,Warrington 001C,E01012476,709,226,31.9
1765,Warrington 001D,E01012477,644,189,29.3
1768,Warrington 002C,E01012475,655,214,32.7
1769,Warrington 002D,E01012554,883,297,33.6
...,...,...,...,...,...
10924,High Peak 002C,E01019730,867,321,37.0
10925,High Peak 002D,E01019731,499,148,29.7
10935,High Peak 004C,E01019754,847,278,32.8
10940,High Peak 005B,E01019747,512,146,28.5


In [25]:
## Merge deprived household percentage to the gdf

merge_lsoa = pd.merge(merge_lsoa, deprived_lsoa[["2021LSOACD", "%"]], left_on="lsoa21cd", right_on="2021LSOACD")
merge_lsoa = merge_lsoa.rename(columns={"%": "deprived_percentage"})

## Drop redundant column
merge_lsoa = merge_lsoa.drop(columns=["2021LSOACD"])

In [26]:
merge_lsoa

,lsoa21cd,lsoa21nm,geometry,area_sqkm,Total,women_percentage,elderly_percentage,children_percentage,pop_density,disabled_no_car,deprived_percentage
0,E01004901,Bolton 028E,"POLYGON ((366003.093 406666.688, 366004.412 40...",0.354363,1773,5.922166,29.606439,15.172025,5003.350923,39,28.7
1,E01005001,Bury 012B,"POLYGON ((375480.348 408593.123, 375481.063 40...",0.238302,1228,4.885993,22.384853,19.218241,5153.129520,14,30.5
2,E01004840,Bolton 012A,"POLYGON ((367374.781 411954.594, 367387.095 41...",4.181101,1769,4.578858,30.275798,19.106840,423.094327,56,32.9
3,E01005006,Bury 012D,"POLYGON ((377212.688 410849.500, 377231.906 41...",1.772635,2051,6.874695,25.688674,20.234032,1157.034831,60,30.5
4,E01004773,Bolton 003D,"POLYGON ((371776.559 412463.204, 371825.243 41...",0.266227,1985,7.103275,25.395631,19.143577,7456.052936,77,36.1
...,...,...,...,...,...,...,...,...,...,...,...
1763,E01006394,Wigan 021B,"POLYGON ((355212.000 403394.000, 355230.000 40...",0.592787,1535,4.560261,34.043014,18.045603,2589.463159,36,33.0
1764,E01006410,Wigan 020E,"POLYGON ((357596.954 403017.375, 357616.740 40...",0.628715,1261,4.758128,33.287718,15.543220,2005.677534,30,35.4
1765,E01006400,Wigan 021D,"POLYGON ((356213.760 403792.397, 356213.312 40...",0.569408,1982,6.004036,18.415945,26.135217,3480.807917,15,26.5
1766,E01006331,Wigan 031D,"POLYGON ((367336.970 401009.452, 367340.657 40...",0.596520,1501,3.997335,28.514146,16.122585,2516.260480,20,27.3


## Export to csv

In [27]:
merge_lsoa.to_csv("Data/lsoa_demand.csv")

## Export to shp

In [28]:
merge_lsoa.to_file("Data/lsoa_demand.shp")

/tmp/ipykernel_15271/491622495.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merge_lsoa.to_file("Data/lsoa_demand.shp")
